In [ ]:
"""
need to find the best libraries for separating speech of the speakers from the video

should try the following:

- native pyannote
- can try if https://replicate.com/lucataco/mvsep-mdx23-music-separation works for speakers as well - 
- https://huggingface.co/speechbrain/sepformer-wham - 
- https://github.com/facebookresearch/svoice - doesnt work cause no inference part provided
- https://github.com/JusperLee/TDANet - 
- https://github.com/modelscope/ClearerVoice-Studio - 

basically want to test each of the repos on my example audio file

might be better if i collect one more sample audio for testing

the question is also how to structuarally represent this somehow - the simplest way is tto make a folder structure like

- audio-separation-samples
    - sample1.wav
    - sample2.wav
    - svoice
        - sample1-result.wav
        - sample2-result.wav
    - pyannote
        - sample1-result.wav
        - sample2-result.wav
    - ...
"""

In [2]:
import soundfile as sf

# Read the audio file
audio_info = sf.info('audio-separation-samples/sample1.wav')
sample_rate_khz = audio_info.samplerate / 1000

print(f"Sample rate: {sample_rate_khz} kHz")

Sample rate: 48.0 kHz


In [7]:
from pydub import AudioSegment

# Load the audio file
audio = AudioSegment.from_wav('audio-separation-samples/sample1.wav')

# Convert to 16kHz
audio_8k = audio.set_frame_rate(8000)

# Export the file
audio_8k.export('audio_8khz.wav', format='wav')

<_io.BufferedRandom name='audio_8khz.wav'>

In [3]:
cd TDANet

/Users/maxhager/projects/pyannote-audio/TDANet


In [4]:
import os
import torch
import look2hear.models
import torchaudio

# Remove or comment out the CUDA device setting since we're using CPU
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# Load and resample audio
mix, sr = torchaudio.load('/Users/maxhager/projects/pyannote-audio/audio-separation-samples/sample1.wav')
transform = torchaudio.transforms.Resample(sr, 16_000)
mix = transform(mix)
mix = mix.view(1, 1, -1)

# Load model with weights_only=False to bypass the security restriction
model = look2hear.models.BaseModel.from_pretrain(
    "JusperLee/TDANetBest-2ms-LRS2", 
    weights_only=False  # Add this parameter
)

# Process audio without .cuda()
est_sources = model(mix)

# Save the separated audio files
torchaudio.save("audio1sep.wav", est_sources[:, 0, :].detach(), 16_000)
torchaudio.save("audio2sep.wav", est_sources[:, 1, :].detach(), 16_000)

In [8]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio

# Initialize model explicitly with CPU
model = separator.from_hparams(
    source="speechbrain/sepformer-wham", 
    savedir='pretrained_models/sepformer-wham',
    run_opts={"device":"cpu"}  # Explicitly set to CPU (you could also just omit this parameter)
)

# Load and separate audio file
est_sources = model.separate_file(path='audio_8khz.wav')

# Save the separated sources
torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach(), 8000)
torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach(), 8000)

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/sepformer-wham' if not cached


hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/sepformer-wham' if not cached
INFO:speechbrain.utils.fetching:Fetch masknet.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wham' if not cached


masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch encoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wham' if not cached


encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch decoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wham' if not cached


decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: masknet, encoder, decoder


In [10]:
import pvfalcon

access_key = "ee4xB9zjbJbfPDjHAuZXkMOsy0IZ/hswOfNTmv1e6u11Ve43L39thA=="

falcon = pvfalcon.create(access_key)

segments = falcon.process_file("output.wav")
for segment in segments:
    print(
        "{speaker_tag=%d start_sec=%.2f end_sec=%.2f}"
        % (segment.speaker_tag, segment.start_sec, segment.end_sec)
    )

{speaker_tag=1 start_sec=0.38 end_sec=3.04}
{speaker_tag=2 start_sec=3.07 end_sec=4.77}
{speaker_tag=3 start_sec=4.86 end_sec=6.56}
{speaker_tag=4 start_sec=6.59 end_sec=9.66}
{speaker_tag=5 start_sec=9.73 end_sec=12.45}
{speaker_tag=5 start_sec=12.93 end_sec=17.18}
{speaker_tag=6 start_sec=17.22 end_sec=21.98}
{speaker_tag=7 start_sec=22.14 end_sec=27.62}
